In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfo = pd.read_sql_table('race', conn)
print(f'{len(dfo)} races loaded!')
dfo = dfo.loc[dfo['race_type'].isin(['R'])]
print(f'{len(dfo)} horse races!')

7609 races loaded!
2542 horse races!


In [3]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,results,runners
1060,1610,RANDWICK,NSW,S,R,2017-09-16,1,TAB HIGHWAY HANDICAP,2017-09-16 02:15:00,Paying,1000,"[[4], [10], [2], [5]]",10.0,"[{""runnerName"": ""AZZIE'S READY"", ""runnerNumber...","[[4], [10], [2], [5]]","[{'runnerName': 'AZZIE'S READY', 'runnerNumber..."
1061,1611,RANDWICK,NSW,S,R,2017-09-16,2,SHOOT OUT MILE,2017-09-16 02:50:00,Paying,1600,"[[3], [5], [8], [7]]",8.0,"[{""runnerName"": ""GOLD HORIZON"", ""runnerNumber""...","[[3], [5], [8], [7]]","[{'runnerName': 'GOLD HORIZON', 'runnerNumber'..."
1062,1612,RANDWICK,NSW,S,R,2017-09-16,3,SCHWEPPES HERITAGE STAKES,2017-09-16 03:25:00,Paying,1100,"[[2], [1], [8], [6]]",8.0,"[{""runnerName"": ""LUCKY LOUIE"", ""runnerNumber"":...","[[2], [1], [8], [6]]","[{'runnerName': 'LUCKY LOUIE', 'runnerNumber':..."
1063,1613,RANDWICK,NSW,S,R,2017-09-16,4,BILL RITCHIE HANDICAP,2017-09-16 04:00:00,Paying,1400,"[[7], [3], [4], [6]]",7.0,"[{""runnerName"": ""PAJARO"", ""runnerNumber"": 6, ""...","[[7], [3], [4], [6]]","[{'runnerName': 'PAJARO', 'runnerNumber': 6, '..."
1064,1614,RANDWICK,NSW,S,R,2017-09-16,5,MCGRATH ESTATE TEA ROSE STAKES,2017-09-16 04:35:00,Paying,1400,"[[6], [2], [5], [1]]",10.0,"[{""runnerName"": ""PANDEMONIUM"", ""runnerNumber"":...","[[6], [2], [5], [1]]","[{'runnerName': 'PANDEMONIUM', 'runnerNumber':..."


In [4]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
#     print(race)

    # add num_runners to all runners
    for runner in race['runners']:
#         print(runner)
        runner['num_runners'] = race['num_runners']
        runner['race_type'] = race['race_type']

    data_all = data_all.append(race['runners'])
    
data_all.tail(10)

,barrierNumber,claimAmount,finishingPosition,fixedOdds,num_runners,odds_perc,odds_scale,odds_win,parimutuel,prediction,probability,race_type,rank_win,riderDriverName,runnerName,runnerNumber,trainerName
2,8,-1.0,0,"{'returnWin': 9.5, 'returnWinOpen': 9.5, 'retu...",12.0,0.105263,0.083230,9.5,"{'bettingStatus': 'Normal', 'returnWin': 14.4,...",0.089557,0.087751,R,5,ADRIE DE VRIES,BOCCA REGIA,9,A NDREAS.SUBORICS
3,3,-1.0,2,"{'returnWin': 16, 'returnWinOpen': 14, 'return...",12.0,0.062500,0.049418,16.0,"{'bettingStatus': 'Normal', 'returnWin': 20.6,...",0.045829,0.044905,R,8,JOZEF BOJKO,IMPOSITO,10,HANS-JURGEN GROSCHEL
4,10,-1.0,0,"{'returnWin': 21, 'returnWinOpen': 19, 'return...",12.0,0.047619,0.037652,21.0,"{'bettingStatus': 'Normal', 'returnWin': 27.2,...",0.034562,0.033865,R,10,ANDREAS HELFENBEIN,PRAIRIE SUNSHINE,11,KARL DEMME
5,5,-1.0,0,"{'returnWin': 41, 'returnWinOpen': 34, 'return...",12.0,0.024390,0.019285,41.0,"{'bettingStatus': 'Normal', 'returnWin': 53.6,...",0.017921,0.017559,R,12,MIGUEL LOPEZ,MATHILDA,12,RAINER KRAPP
6,1,-1.0,0,"{'returnWin': 19, 'returnWinOpen': 12, 'return...",12.0,0.052632,0.041615,19.0,"{'bettingStatus': 'Normal', 'returnWin': 53.6,...",0.038581,0.037803,R,9,SONJA DAROSZEWSKI,NAREIA,1,J OHN.WARREN
7,11,-1.0,1,"{'returnWin': 9, 'returnWinOpen': 8.5, 'return...",12.0,0.111111,0.087854,9.0,"{'bettingStatus': 'Normal', 'returnWin': 16.9,...",0.098276,0.096294,R,4,BAYARSAIKHAN GANBA,DASHING DIVA,2,STEFAN RICHTER
8,6,-1.0,0,"{'returnWin': 12, 'returnWinOpen': 11, 'return...",12.0,0.083333,0.065890,12.0,"{'bettingStatus': 'Normal', 'returnWin': 17.4,...",0.062495,0.061235,R,7,ALEXANDER PIETSCH,ZITAT,3,STEFAN RICHTER
9,12,-1.0,3,"{'returnWin': 4, 'returnWinOpen': 4.6, 'return...",12.0,0.250000,0.197671,4.0,"{'bettingStatus': 'Normal', 'returnWin': 5.4, ...",0.185164,0.181430,R,1,MARC LERNER,RICH ROOFER,4,AXEL KLEINKORRES
10,4,-1.0,0,"{'returnWin': 8, 'returnWinOpen': 7.5, 'return...",12.0,0.125000,0.098835,8.0,"{'bettingStatus': 'Normal', 'returnWin': 9.8, ...",0.138914,0.136112,R,3,FILIP MINARIK,ANTONIUS,5,HANS-JURGEN GROSCHEL
11,9,-1.0,0,"{'returnWin': 9.5, 'returnWinOpen': 9.5, 'retu...",12.0,0.105263,0.083230,9.5,"{'bettingStatus': 'Normal', 'returnWin': 11.1,...",0.089557,0.087751,R,5,MAXIM PECHEUR,ATLAS ROYAL,6,HEINRICH NIENSTADT


In [5]:
# drop scratched
data = data_all.dropna(subset=['odds_win'])
data = data[(data.odds_win > 0)]
data.describe()

,barrierNumber,claimAmount,finishingPosition,num_runners,odds_perc,odds_scale,odds_win,prediction,probability,rank_win,runnerNumber
count,25787.000000,25787.000000,25787.000000,25787.000000,25787.000000,25787.000000,25787.000000,25787.000000,25787.000000,25787.000000,25787.000000
mean,5.855664,-0.361655,0.828596,10.455113,0.130295,0.098577,21.484950,0.097682,0.098577,5.908908,6.320123
std,3.591578,1.275484,1.281039,2.952117,0.136023,0.095704,29.380591,0.102250,0.100649,3.634449,3.879648
min,0.000000,-1.000000,0.000000,3.000000,0.003322,0.000778,1.010000,0.000000,0.000000,1.000000,1.000000
25%,3.000000,-1.000000,0.000000,8.000000,0.043478,0.031675,5.500000,0.028526,0.028470,3.000000,3.000000
50%,5.000000,-1.000000,0.000000,10.000000,0.090909,0.068101,11.000000,0.063240,0.064240,5.000000,6.000000
75%,8.000000,0.000000,2.000000,12.000000,0.181818,0.134576,23.000000,0.136513,0.139888,8.000000,9.000000
max,24.000000,5.000000,4.000000,22.000000,0.990099,0.893369,301.000000,1.000000,0.999924,23.000000,24.000000


In [6]:
# get label data
Y = data['finishingPosition'] == 1
Y = Y.astype(int)
Y.describe()

count    25787.000000
mean         0.098848
std          0.298464
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: finishingPosition, dtype: float64

In [7]:
# xp = fixed odds perc
xp = data['odds_perc']
xp.describe()

count    25787.000000
mean         0.130295
std          0.136023
min          0.003322
25%          0.043478
50%          0.090909
75%          0.181818
max          0.990099
Name: odds_perc, dtype: float64

In [8]:
# xs = odds scaled
xs = data['odds_scale']
xs.describe()

count    25787.000000
mean         0.098577
std          0.095704
min          0.000778
25%          0.031675
50%          0.068101
75%          0.134576
max          0.893369
Name: odds_scale, dtype: float64

In [9]:
# xr = win rank
xr = data['rank_win']
xr.describe()

count    25787.000000
mean         5.908908
std          3.634449
min          1.000000
25%          3.000000
50%          5.000000
75%          8.000000
max         23.000000
Name: rank_win, dtype: float64

In [10]:
# xn = get num runners
xn = data['num_runners']
xn.describe()

count    25787.000000
mean        10.455113
std          2.952117
min          3.000000
25%          8.000000
50%         10.000000
75%         12.000000
max         22.000000
Name: num_runners, dtype: float64

In [11]:
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

,odds_perc,odds_scale,rank_win,num_runners
count,25787.000000,25787.000000,25787.000000,25787.000000
mean,0.130295,0.098577,5.908908,10.455113
std,0.136023,0.095704,3.634449,2.952117
min,0.003322,0.000778,1.000000,3.000000
25%,0.043478,0.031675,3.000000,8.000000
50%,0.090909,0.068101,5.000000,10.000000
75%,0.181818,0.134576,8.000000,12.000000
max,0.990099,0.893369,23.000000,22.000000


In [12]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 1000
print('epochs = {}'.format(epochs))

tag = 'R30x30'
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=n))
    model.add(Dense(30, activation='relu'))
#     model.add(Dense(40, activation='relu'))
#     model.add(Dense(40, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

input dimension = 4
epochs = 1000


In [13]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)